# Actividad 1: Regresion

Esta actividad debe ser completada antes del Viernes de esta semana, a las 18:30 horas. Deberás subir el notebook completo y reportar tu nivel de avance en el cuestionario habilitado en canvas. 

La meta, por supuesto, es aprender a estimar un modelo de regresión lineal. Comenzemos por cargar datos para armar una regresion. Esta vez nuestros datos son tabulares, pero pandas los puede leer sin mucho problema (están limpios). Los datos corresponden al dataset usado por Francis Galton para estudiar la estatura de las personas (de este ejercicio, publicado en 1886, viene el término 'regresión'). 

In [1]:
import pandas as pd
galton = pd.read_table('galton-stata11.tab')
galton.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 898 entries, 0 to 897
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   family  898 non-null    object 
 1   father  898 non-null    float64
 2   mother  898 non-null    float64
 3   gender  898 non-null    object 
 4   height  898 non-null    float64
 5   kids    898 non-null    int64  
 6   male    898 non-null    float64
 7   female  898 non-null    float64
dtypes: float64(5), int64(1), object(2)
memory usage: 56.2+ KB


'family' es un identificador asignado a cada familia. 'father' y 'mother' es la estatura de su padre y madre. 'height' es la estatura del hijo (una vez adulto). 'gender', 'male' y 'female' son autoexplicativos. 

In [3]:
galton

,family,father,mother,gender,height,kids,male,female
0,1,1.9939,1.7018,M,1.85928,4,1.0,0.0
1,1,1.9939,1.7018,F,1.75768,4,0.0,1.0
2,1,1.9939,1.7018,F,1.75260,4,0.0,1.0
3,1,1.9939,1.7018,F,1.75260,4,0.0,1.0
4,2,1.9177,1.6891,M,1.86690,4,1.0,0.0
...,...,...,...,...,...,...,...,...
893,136A,1.7399,1.6510,M,1.73990,8,1.0,0.0
894,136A,1.7399,1.6510,M,1.71958,8,1.0,0.0
895,136A,1.7399,1.6510,F,1.62560,8,0.0,1.0
896,136A,1.7399,1.6510,F,1.61290,8,0.0,1.0


Las alturas están en pulgadas, lo que es horrible. Las pasamos a metros

In [2]:
galton['father'] = 0.0254 * galton['father']
galton['mother'] = 0.0254 * galton['mother']
galton['height'] = 0.0254 * galton['height']

### Ajustando una regresión

Nota que tenemos que sacar la variable 'family' y la de 'gender', por que no son números. La de gender la tenemos con male/female, y el id de familia no debería aportarnos mucho acá, así que lo sacamos sin cuidado. 

In [4]:
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score

# Crea un objeto regresión
regr = linear_model.LinearRegression()

# Lo entrenamos usando el dataset de Galton, queremos predecir la altura (height)
regr.fit(galton[['father','mother','kids','male','female']], galton['height'])

# Imprimimos los coeficientes (los beta). Están en el orden en que están los atributos
print('Coeficientes: \n', regr.coef_)
print('Beta0: \n', regr.intercept_)


Coeficientes: 
 [ 3.98305279e-01  3.22967372e-01 -1.10721125e-03  2.16341597e+11
  2.16341597e+11]
Beta0: 
 -216341596568.7178


Y bueno, ya tenemos una regresión. ¿Qué podemos hacer? podemos, por ejemplo, predecir cuando mediría una niña dada la altura de su papá, de su mamá y cuantos hijos tuvieron sus padres.  

In [5]:
#Diccionario con dos filas, papá y mamá miden dos metros, tienen dos hijos, uno es hombre y otro mujer
para_predecir = pd.DataFrame({'father':[2,2],'mother':[2,2],'kids':[2,2],'male':[1,0],'female':[0,1]})
print(para_predecir)

#ahora le pedimos al modelo de regresión que nos prediga la estatura de los hijos: 
regr.predict(para_predecir)

#el resultado: hijo hombre mediría 1.98, hija mujer 1.84

   father  mother  kids  male  female
0       2       2     2     1       0
1       2       2     2     0       1


array([1.98071289, 1.84793091])

Ahora bien. Hay un campo que está un poco extraño, y es la cantidad de hijos que tiene esta familia. Por qué debería influir la cantidad de hermanos en la altura de una persona? ¿Qué pasa si en vez de predecir el valor de arriba para dos hijos, predecirmos como si la familia tuviera 7 hijos?

In [6]:
para_predecir = pd.DataFrame({'father':[2,2],'mother':[2,2],'kids':[17,17],'male':[1,0],'female':[0,1]})
print(para_predecir)


regr.predict(para_predecir)

   father  mother  kids  male  female
0       2       2    17     1       0
1       2       2    17     0       1


array([1.96411133, 1.83132935])

El resultado cambia un poco! Esto no hace ningún sentido: ¿cómo va a ser que la altura de las personas dependa de la cantidad de sus hermanos? Algo extraño debe estar pasando. 

### Ejercicio 1

Entrena una regresión que solo tome en cuenta 'father', 'mother', 'male', 'female', y luego una que solo tome en cuenta 'mother', 'male', 'female'. Predice valores como los de arriba, y discute sobre cuál es mejor. 

# Seleccionando mejores modelos

Antes de entender qué pasa que nuestro modelo toma en cuenta la cantidad de hermanos a la hora de predecir, veamos algo más general: ¿cómo seleccionar los mejores modelos?

## Alternativa 1: Mean-squared error (MSE), r^2

Recordemos que otra forma de ajustar una regresión es encontrar una función que minimiza los cuadrados de los errores: 
en nuestro caso el error de cada punto se calcula como el valor de height en el dataset menos el valor predicho por la regresión, todo eso al cuadrado:  Si tomamos la media de ese error, llegamos al MSE. Lógicamente, mientras mas chico el MSE, más se ajusta la función a los datos. 

$$\text{MSE} = \frac{1}{n} \sum_{1 \leq i \leq n} (y_i - \beta^T\bar x^i)^2$$

El coeficiente r^2 corresponde a la suma de los errores, dividido por la diferencia entre cada punto y su media. 

$$r^2 = 1 - \frac{\sum_{1 \leq i \leq n} (y_i - \beta^T\bar x^i)^2}{\sum_{1 \leq i \leq n} (y_i - \bar y)^2},$$

donde $$\bar y = \frac{\sum_{1 \leq i \leq n} y_i}{n}$$

El coeficiente r^2 va desde 1 (cuando la suma de los errores es cero, un modelo que se ajusta perfecto) a 0 (cuando la suma de los errores es igual a la diferencia entre cada punto y su media, el modelo no sirve para nada). 

Veamos como obtener el MSE, y el coeficiente r^2. 



In [7]:
#pedimos la predicción de todos los datos. 
galton_pred = regr.predict(galton[['father','mother','kids','male','female']])

#calculamos el MSE
print('Mean squared error: %f'
      % mean_squared_error( galton['height'], galton_pred))

#calculamos el r^2
print('Coeficiente r2: %.2f'
      % r2_score(galton['height'], galton_pred))

Mean squared error: 0.002972
Coeficiente r2: 0.64


### Ejercicio 2

¿Cuál de los tres modelos que has entrenado tiene mejor MSE? ¿mejor r^2?

## Overfitting

Si hiciste el **Ejercicio 2**, quizás te estarás preguntando por qué modelos con la variable 'kids' funcionan mejor que algunos sin esa variable, siendo que no debería tener nada que ver en este caso. 

La respuesta a este tipo de preguntas muchas veces viene del hecho que los modelos de machine learning terminan aprendiendo *demasiado* los datos, hasta un punto en que se aprovechan de patrones espureos para poder ajustar un poquitito mejor la recta. En este caso, bien puede haber pasado algo así. 

¿Cuál es el problema del overfitting? Que cuando queramos usar nuestro predictor en un ambiente real, los datos quizá no van a respetar esa correlación, y nuestro modelo va a terminar siendo, en la práctica, peor. Una forma bien estándar de asegurarnos que esto no pasa es dividir el set de datos, de forma de calcular el MSE con datos que son distintos a los que se usaron para entrenar.  

### Alternativa 2:  Train-test split

La idea es simple: vamos a ajustar la regresión con menos datos de los que teníamos (digamos un 80% de los datos), y vamos a *reservar* los otros para calcular el MSE y el r^2 con datos realistas, que no se usaron para entrenar al modelo.

In [11]:
from sklearn.model_selection import train_test_split

galton_train, galton_test = train_test_split(galton, train_size=0.80)
print(galton_train.count())
print(galton_test.count())

family    718
father    718
mother    718
gender    718
height    718
kids      718
male      718
female    718
dtype: int64
family    180
father    180
mother    180
gender    180
height    180
kids      180
male      180
female    180
dtype: int64


Ahora usamos nuestro split: Entrenamos con el set de train **galton_train**, y cuando vayamos a ver que tan bueno es el modelo (según MSE, r^2), lo vamos a hacer usando el test. 

In [12]:
regr = linear_model.LinearRegression()

# Solo usamos el set de train
regr.fit(galton_train[['father','mother','kids','male','female']], galton_train['height'])

#ahora usamos el modelo regr pa predecir la altura según el set de test
galton_pred = regr.predict(galton_test[['father','mother','kids','male','female']])

#y calculamos 
print('Mean squared error: %f'
      % mean_squared_error( galton_test['height'], galton_pred))

print('Coeficiente r2: %.2f'
      % r2_score(galton_test['height'], galton_pred))

Mean squared error: 0.002917
Coeficiente r2: 0.63


Comparemos con el modelo sin el numero de hermanos. Vemos que el MSE ahora es ligeramente inferior, por lo que nos convendría más este modelo. ¿Cómo se compara esto con la situación anterior, cuando no hacíamos **train/test split**?

In [13]:
regr = linear_model.LinearRegression()

# Solo usamos el set de test
regr.fit(galton_train[['father','mother','male','female']], galton_train['height'])

#ahora usamos el modelo regr pa predecir la altura según el set de test
galton_pred = regr.predict(galton_test[['father','mother','male','female']])

#y calculamos 
print('Mean squared error: %f'
      % mean_squared_error( galton_test['height'], galton_pred))

print('Coeficiente r2: %.2f'
      % r2_score(galton_test['height'], galton_pred))

Mean squared error: 0.002910
Coeficiente r2: 0.64


### K-fold cross validation

A veces tenemos mala suerte y el pedazo de test justo se lleva algunos datos muy importantes para la regresión. Para evitar esto, una mejor práctica es hacer variso splits distintos. Esto se conoce como k-fold cross validation, en donde k es el número de grupos. 

La práctica de k-fold cross validation consiste en: 
- particionar los datos en k grupos distintos, todos iguales
- entrenar k modelos distintos: el modelo j-ésimo usa para entrenar todos los grupos **menos** el grupo j, y usa el grupo j para testear. 
- los indicadores de cada modelo se agregan en un número. En nuestro caso, vamos a computar el MSE y el r^2, y el total va a ser simplemente el promedio de los k. 

### Ejercicio 3

Implementa 5-fold cross validation. Prueba con los tres modelos que vimos, y calcula el MSE y el r^2 total para cada uno de ellos. 

### Qué k es mejor usar? 

Usualmente se trabaja con k = 5 o k = 10, aunque algunas veces tiene sentido hacer que k se el tamaño del dataset (eso se conoce como "leave one out cross validation"). Ante la duda, ¡probar con k = 5 y k = 10! 

### Alternativa 3: Regularizar con LASSO. 

Otra alternativa para reducir el overfitting es poner una penalización en la función de verosimilitud a la hora de maximizarla. Es decir, vamos a maximizarla, pero vamos a incluir una penalización que depende de la cantidad coeficientes. Esto va a *empujar* a que algunos de esos coeficientes sean 0, cuando la diferencia entre la verosimilitud con/sin esos coeficientes es pequeña. 

Nosotros solo vamos a ver LASSO, que corresponde a penalizar con el valor absoluto de cada coeficiente: para una regresión sobre entidades con $n$ atributos, la verosimilitud penalizada $\mathcal P(\bar x \mid p)$ se calcula como 
$$\mathcal P(\bar x \mid p) = \mathcal L(\bar x \mid p) - \lambda\sum_{1 \leq i \leq n}|\beta_i|.$$

Acá, $\lambda$ es un parámetro que debe ser proveído por el cientista de datos a cargo de entrenar el modelo (aunque podemos buscar el mejor $\lambda$)

Una cosa importante: Como LASSO suma los coeficientes, es importante **estandarizar** nuestros datos. Para eso, tomamos vada valor v y lo transformamos en $(v-\text{media})/ (\text{desviacion estandar})$. Nauralmente, no hacemos esto con las variables indicatrices (*male*, *female*) por que esos datos ya vienen estandarizados. 

In [14]:
galton_std = galton[['father','mother','kids','male','female','height']].copy()
for column in galton_std.columns:
   if column != 'male' and column != 'female':
        galton_std[column] = (galton_std[column] - galton_std[column].mean()) / galton_std[column].std()

galton_std.head()

,father,mother,kids,male,female,height
0,3.751494,1.263788,-0.795431,1.0,0.0,1.797225
1,3.751494,1.263788,-0.795431,0.0,1.0,0.680816
2,3.751494,1.263788,-0.795431,0.0,1.0,0.624996
3,3.751494,1.263788,-0.795431,0.0,1.0,0.624996
4,2.537045,1.047058,-0.795431,1.0,0.0,1.880955


Recuerda que el significado de estos números en versión estandarizada indica cuantas desviaciones estándar están desde la media (positivo es más a la derecha de la media, negativo es a la izquierda). 

Entrenemos ahora una regresión con LASSO. 

In [15]:
#Entrenamos una regresión con lasso, usamos un alfa de 0.05
regr_regularizado = linear_model.Lasso(alpha=0.05)

regr_regularizado.fit(galton_std[['father','mother','kids','male','female']], galton_std['height'])

print(regr_regularizado.coef_)
print(regr_regularizado.intercept_)

regr_regularizado.predict(para_predecir)

#Ahora predecimos un poco y calculamos el MSE y r^2
galton_pred = regr_regularizado.predict(galton_std[['father','mother','kids','male','female']])

#y calculamos 
print('Mean squared error: %f'
      % mean_squared_error( galton_std['height'], galton_pred))

print('Coeficiente r2: %.2f'
      % r2_score(galton_std['height'], galton_pred))



[ 2.30736471e-01  1.56898288e-01 -0.00000000e+00  1.25207987e+00
 -5.38734282e-16]
-0.6483487067255513
Mean squared error: 0.375212
Coeficiente r2: 0.62


Fijate como ha desaparecido el coeficiente correspondiente a *kids*, y como casi desaparece el de *female* (este es superfluo, basta con uno entre male/female. Es por este motivo que LASSO tiende a regularizar de forma práctica: los valores de los coeficientes van bajando hasta transformarse en 0. 

Lo otro que es interesante: fíjate como ha subido el coeficiente r^2. El MSE no es comparable con el de la regresión que no está estandarizada. 

### Ejercicio 4

Busca el mejor $\lambda$ para usar LASSO. PAra ver cuál modelo es mejor, usa 5-fold cross validation, y selecciona los modelos que tengan el menor MSE (o el menor r^2, como prefieras). 